<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
from imblearn.combine import SMOTETomek

y_train_smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = y_train_smt.fit_resample(X_train, y_train)
np.unique(y_train_smt, return_counts=True)

(array([SMOTETomek(random_state=42)], dtype=object), array([1]))

### Track Experiments

In [6]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [7]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [13]:
# Initialize MLflow
import mlflow

# ✅ ALWAYS set tracking URI first
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Anomaly Detection v2")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):

        # Log parameters
        mlflow.log_params(params)

        # Log metrics
        mlflow.log_metrics({
            "accuracy": report["accuracy"],
            "recall_class_1": report["1"]["recall"],
            "recall_class_0": report["0"]["recall"],
            "f1_score_macro": report["macro avg"]["f1-score"]
        })

        # ✅ SAFE for all models (XGB, RF, LR, etc.)
        mlflow.sklearn.log_model(model, "model")


2025/12/23 16:23:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 16:23:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/3/runs/bef5293e6cd74597b76733b9aff366f3
🧪 View experiment at: http://localhost:5000/#/experiments/3
🏃 View run Random Forest at: http://localhost:5000/#/experiments/3/runs/80c796c496c942e58f496f8ff4c4c666
🧪 View experiment at: http://localhost:5000/#/experiments/3


2025/12/23 16:23:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/23 16:24:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/3/runs/c965b67fd8d84e03a2f14fbbfd88ace0
🧪 View experiment at: http://localhost:5000/#/experiments/3
🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/3/runs/a8444b67f3124096bf2719a7c134eff4
🧪 View experiment at: http://localhost:5000/#/experiments/3


### Register the Model

In [16]:
import mlflow

model_name = "XGB-Smote"
run_id = input("Enter Run ID: ")

# ✅ Must EXACTLY match log_model path
model_uri = f"runs:/{run_id}/model"

# ✅ No start_run() needed here
mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)

print("Model registered successfully!")


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/12/23 16:33:32 WARNING mlflow.tracking._model_registry.fluent: Run with id a8444b67f3124096bf2719a7c134eff4 has no artifacts at artifact path 'model', registering model based on models:/m-16298def79f7485e94e9b34daa5afffb instead
2025/12/23 16:33:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


Model registered successfully!


Created version '1' of model 'XGB-Smote'.


### Load the Model

In [18]:
import mlflow

model_name = "XGB-Smote"
model_version = 1

model_uri = f"models:/{model_name}/{model_version}"

# ✅ Load with sklearn flavor
loaded_model = mlflow.sklearn.load_model(model_uri)

# Predict
y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

### Transition the Model to Production

In [19]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1766488477701, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1766488477701, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='a8444b67f3124096bf2719a7c134eff4', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [22]:
import mlflow

prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.sklearn.load_model(prod_model_uri)

y_pred = loaded_model.predict(X_test)
y_pred[:4]


array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 